In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [3]:
x = torch.tensor([
    [0,0],
    [0,1],
    [1,1]
])

In [4]:
def method1(x):
    xx = F.pdist(x)
    m = torch.zeros((x.shape[0],x.shape[0]))
    triu_indices = torch.triu_indices(row=x.shape[0], col=x.shape[0], offset=1)
    m[triu_indices[0], triu_indices[1]] = xx
    m[triu_indices[1], triu_indices[0]] = xx
    return m

def method2(x):
    return torch.norm(x[:, None] - x, dim=2, p=2)

# def method1(x):
#     xx = F.pdist(x)
#     m = torch.zeros((x.shape[0],x.shape[0]))
#     triu_indices = torch.triu_indices(row=x.shape[0], col=x.shape[0], offset=1)
#     m[triu_indices[0], triu_indices[1]] = xx
#     return (m + m.T).sort(dim=1)

In [5]:
x = torch.rand((100,2))

In [6]:
%timeit method1(x)


121 µs ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
%timeit method2(x)

1.85 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
torch.norm(method1(x)[:,:] - method2(x)[:,:])

tensor(0.)

In [9]:
model = nn.Embedding(100,128)

print(sum(t.numel() for t in model.parameters() if t.requires_grad))

model = nn.Linear(100,128)

print(sum(t.numel() for t in model.parameters() if t.requires_grad))

12800
12928


In [62]:
pi = torch.tensor([
    [0,2,1],
    [2,1,0]
])
# dataset = torch.tensor([
#     [
#         [1,1.],
#         [2,2],
#         [3,3]
#     ],
#     [
#         [-1,-1],
#         [-2,-2],
#         [-3,-3]
#     ]
# ])

for _ in range(10000):
    dataset = torch.rand((2,3,2))
    d = dataset.gather(1, pi.unsqueeze(-1).expand_as(dataset))
    res = (d[:, 1:] - d[:, :-1]).norm(p=2, dim=2).sum(1) + (d[:, 0] - d[:, -1]).norm(p=2, dim=1)

    # metodo con distanze
    dataset_dist = torch.stack(tuple(method1(d) for d in dataset),0)
    a = torch.arange(pi.shape[1])
    idx = torch.stack((a, a.roll(-1,0)))
    res_dist = dataset_dist[:,idx[0],idx[1]].sum(1)

    assert torch.norm(res - res_dist) <= 1e-6, f'{res} {res_dist}'